## Reading Input

In [1]:
import pandas as pd
from proj_utils.input_interpreter import format_input

In [2]:
customer_df = pd.read_excel('data/2_detail_table_customers.xls')
deposit_df = pd.read_excel('data/4_detail_table_depots.xls')
routes = customer_df['ROUTE_ID'].unique()

In [3]:
print(routes)

[2946091 2922001 2939484 2958047 2970877 2990001 3005971 2604001 3016355
 3027038 3044702]


In [4]:
from proj_utils.fake_input_generator import generator
from proj_utils.input_interpreter import format_input

import matplotlib.pyplot as plt

from VRP.greedy_solutions import nearest_neighbour_solution
from VRP.modify_solution import combined_rand_modification
from VRP.verify_solution import valid_path, calculate_path_distance

from proj_utils.path import print_path

from solver_step import SolverStep
%matplotlib widget

# Simulated Annealing Solution

In [5]:
import simulated_annealing.solver as SA # Simulated anealing solver

In [6]:
#dists, demand, points = format_input(routes[2],customer_df,deposit_df)
dists, demand, points = generator(20)

In [7]:
solver_step = SA.generic_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    lambda x: calculate_path_distance(x, dists) if(valid_path(x,(30,20),demand)) else float('inf'),
    )

In [8]:
init_state = nearest_neighbour_solution(dists, demand, (30,20))
init_step = SolverStep(0,init_state,2)
step_1 = solver_step(init_step)
print(step_1.get_best_sol())

[0, 1, 9, 14, 11, 18, 17, 7, 2, 8, 6, 15, 16, 13, 10, 3, 19, 5, 12, 4, 0]


In [9]:
step_1.set_step_size(300)
step_2 = solver_step(step_1)
print(step_2.get_best_sol())

[0, 9, 1, 14, 6, 2, 8, 15, 16, 13, 10, 3, 17, 19, 7, 18, 11, 4, 12, 5, 0]


# Taboo Search

In [10]:
import taboo_search.solver as TS

In [11]:
dists, demand, points = generator(20)

In [12]:
taboo_list_size = 10
solver_step = TS.taboo_search_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    lambda x: calculate_path_distance(x, dists) if(valid_path(x,(30,20),demand)) else float('inf'),
    taboo_list_size
    )

In [13]:
init_state = nearest_neighbour_solution(dists, demand, (30,20))
init_step = SolverStep(0,init_state,2)
step_1 = solver_step(init_step)
print(step_1.get_best_sol())

[0, 1, 2, 14, 12, 16, 4, 9, 5, 8, 18, 13, 15, 3, 17, 10, 6, 19, 11, 7, 0]


In [14]:
step_1.set_step_size(300)
step_2 = solver_step(step_1)
print(step_2.get_best_sol())

[0, 11, 7, 19, 13, 18, 5, 14, 9, 16, 4, 12, 2, 1, 3, 10, 17, 6, 15, 8, 0]
